In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display
import librosa.display
import numpy as np
import librosa
import tensorflow as tf
import glob

C:\Users\Jaesung\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#uav_path = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/JUNE_01_PHANTOMS/P1_stationary.wav'
#none_path = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/JUNE_02_BACKGROUND/background_06_02_01.wav'

uav_path = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/JUNE_01_PHANTOMS/*.wav'
none_path = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/JUNE_02_BACKGROUND/background_0*.wav'

#uav_path = '../../../RW_AUDIO_DATA_2018_Update/JUNE_01_PHANTOMS/*.wav'
#none_path = '../../../RW_AUDIO_DATA_2018_Update/JUNE_02_BACKGROUND/background_0*.wav'

uav_files = glob.glob(uav_path)
none_files = glob.glob(none_path)
#print(uav_files)
#print(none_files)

In [3]:
CHUNK_SIZE = 8192
SR = 44100
N_MFCC = 16

In [4]:
def load(files, sr=44100):
    [raw, sr] = librosa.load(files[0], sr=sr)
    for f in files[1:]:
        [array, sr] = librosa.load(f, sr=sr)
        raw = np.hstack((raw, array))
    print(raw.shape)
    return raw

In [5]:
uav_raw = load(uav_files)
none_raw = load(none_files)

(9732096,)
(18579456,)


# Data preprocessing

Chose `mfcc4` among functions below:
- input size 16x16; `n_mfcc=16`, used first 16 data points from 16x17 mfcc of a chunk with size 8192
- chunk siez 8192, overlapped half of it

In [6]:
def mfcc4(raw, label, chunk_size=8192, window_size=4096, sr=44100, n_mfcc=16, n_frame=16):
    mfcc = np.empty((0, n_mfcc, n_frame))
    y = []
    print(raw.shape)
    for i in range(0, len(raw), chunk_size//2):
        mfcc_slice = librosa.feature.mfcc(raw[i:i+chunk_size], sr=sr, n_mfcc=n_mfcc) #n_mfcc,17
        if mfcc_slice.shape[1] < 17:
            print("small end:", mfcc_slice.shape)
            continue
        mfcc_slice = mfcc_slice[:,:-1]
        mfcc_slice = mfcc_slice.reshape((1, mfcc_slice.shape[0], mfcc_slice.shape[1]))
        mfcc = np.vstack((mfcc, mfcc_slice))
        y.append(label)
    y = np.array(y)
    return mfcc, y

In [7]:
mfcc_uav, y_uav = mfcc4(uav_raw, 1)
print(mfcc_uav.shape, y_uav.shape)
mfcc_none, y_none = mfcc4(none_raw, 0)
print(mfcc_none.shape, y_none.shape)


(9732096,)
small end: (16, 9)
(2375, 16, 16) (2375,)
(18579456,)
small end: (16, 9)
(4535, 16, 16) (4535,)


In [8]:
print(mfcc_uav.shape, y_uav.shape) ###### 
print(mfcc_none.shape, y_none.shape) ######    

(2375, 16, 16) (2375,)
(4535, 16, 16) (4535,)


In [9]:
X = np.concatenate((mfcc_uav, mfcc_none), axis=0)
y = np.hstack((y_uav, y_none))
print(X.shape, y.shape)

(6910, 16, 16) (6910,)


In [10]:
n_labels = y.shape[0]
n_unique_labels = 2
y_encoded = np.zeros((n_labels, n_unique_labels))
y_encoded[np.arange(n_labels), y] = 1
print(y_encoded.shape)

(6910, 2)


In [11]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [12]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(5528, 16, 16) (1382, 16, 16)
(5528, 2) (1382, 2)


In [13]:
### path selection between jaesung & gunhoo ###
np.save('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/X_train_chunk_3d', X_train)
np.save('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/X_test_chunk_3d', X_test)
np.save('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/y_train_chunk_3d', y_train)
np.save('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/y_test_chunk_3d', y_test)
#np.save('../../../RW_AUDIO_DATA_2018_Update/Xy/X_train_chunk_3d', X_train)
#np.save('../../../RW_AUDIO_DATA_2018_Update/Xy/X_test_chunk_3d', X_test)
#np.save('../../../RW_AUDIO_DATA_2018_Update/Xy/y_train_chunk_3d', y_train)
#np.save('../../../RW_AUDIO_DATA_2018_Update/Xy/y_test_chunk_3d', y_test)

In [14]:
### path selection between jaesung & gunhoo ###
X_train = np.load('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/X_train_chunk_3d.npy')
X_test = np.load('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/X_test_chunk_3d.npy')
y_train = np.load('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/y_train_chunk_3d.npy')
y_test = np.load('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/y_test_chunk_3d.npy')
#X_train = np.load('../../../RW_AUDIO_DATA_2018_Update/Xy/X_train_chunk_3d.npy')
#X_test = np.load('../../../RW_AUDIO_DATA_2018_Update/Xy/X_test_chunk_3d.npy')
#y_train = np.load('../../../RW_AUDIO_DATA_2018_Update/Xy/y_train_chunk_3d.npy')
#y_test = np.load('../../../RW_AUDIO_DATA_2018_Update/Xy/y_test_chunk_3d.npy')
#np.save('../../../RW_AUDIO_DATA_2018_Update/Xy/y_train_chunk_3d', y_train)
#np.save('../../../RW_AUDIO_DATA_2018_Update/Xy/y_test_chunk_3d', y_test)

# Tensorflow finally!

## Training

### Parameters
- input data: ?, 16, 16, 1 (batch size, #mfcc, #frame, #channels)
- filter: [3, 3]
- strides: 1(first layer), 2(second layer)

# Experiment 3 - One convolutional layer /w no dropout

#### Experiment 3-2
- learning rate 0.005
- pooling stride 1x1
- #filter 1
- best result among every other settings
- cost kept fluctuated during training. (0.8 -> 1.3) -- why is that?

In [26]:
tf.reset_default_graph()

In [51]:
n_mfcc = 16
n_frame = 16
n_classes = 2
n_channels = 1

kernel_size = 3
stride = 1
pad = "SAME"

learning_rate = 0.005
training_epochs = 20

In [52]:
X = tf.placeholder(tf.float32, shape=[None,n_mfcc*n_frame*n_channels])
X = tf.reshape(X, [-1, n_mfcc, n_frame, n_channels])
Y = tf.placeholder(tf.float32, shape=[None,n_classes])


conv1 = tf.layers.conv2d(inputs=X, filters=1, kernel_size=[3, 3],
                         padding="SAME", activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                                padding="SAME", strides=1)
#dropout1 = tf.layers.dropout(inputs=pool1, rate=0.7, training=True)

flat = tf.reshape(pool1, [-1, 16*16*1])

In [53]:
dense2 = tf.layers.dense(inputs=flat, units=625, activation=tf.nn.relu)
#dropout2 = tf.layers.dropout(inputs=dense2, rate=0.5, training=True)
logits = tf.layers.dense(inputs=dense2, units=2)

In [54]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [55]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [32]:
X_train2 = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test2 = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

In [33]:
model_path = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/GIt/uav-audio-detection/models/CNN/my_test_model_cnn'
#saver = tf.train.Saver()
#model_path = './model_cnn.ckpt'
#model_path = '../my_test_model_cnn'
saver = tf.train.Saver()

In [34]:
batch_size = 100
cost_history = np.empty(shape=[1], dtype=float)

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(y_train.shape[0] / batch_size)
    for i in range(0, y_train.shape[0], batch_size):
        feed_dict={X:X_train2[i:i+batch_size,:,:,:], Y:y_train[i:i+batch_size,:]}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        cost_history = np.append(cost_history,cost)
        avg_cost += c/total_batch
    print('Epoch:', '%04d' % (epoch+1), 'cost = ', '{:.9f}'.format(avg_cost))
saver.save(sess, model_path)

Epoch: 0001 cost =  27.346735285
Epoch: 0002 cost =  1.316307159
Epoch: 0003 cost =  0.690751313
Epoch: 0004 cost =  0.391502521
Epoch: 0005 cost =  0.352517903
Epoch: 0006 cost =  0.352861932
Epoch: 0007 cost =  0.350578950
Epoch: 0008 cost =  0.316064002
Epoch: 0009 cost =  0.275687385
Epoch: 0010 cost =  0.279054859
Epoch: 0011 cost =  0.176479019
Epoch: 0012 cost =  0.218926024
Epoch: 0013 cost =  0.136181130
Epoch: 0014 cost =  0.098120852
Epoch: 0015 cost =  0.075162654
Epoch: 0016 cost =  0.051261087
Epoch: 0017 cost =  0.064446117
Epoch: 0018 cost =  0.045772156
Epoch: 0019 cost =  0.041995593
Epoch: 0020 cost =  0.039950972


'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/GIt/uav-audio-detection/models/CNN/my_test_model_cnn'

In [35]:
y_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_test2})
y_true = sess.run(tf.argmax(y_test,1))

In [36]:
from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_true, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

F-Score: 0.975
Accuracy:  0.975397973950796
             precision    recall  f1-score   support

          0       0.96      1.00      0.98       885
          1       1.00      0.93      0.96       497

avg / total       0.98      0.98      0.98      1382

[[885   0]
 [ 34 463]]


###########################################################3

### 모델불러오기

In [37]:
uav_path = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/JUNE_01_PHANTOMS/P1_stationary.wav'
none_path = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/JUNE_02_BACKGROUND/background_06_02_01.wav'

#uav_path = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/JUNE_01_PHANTOMS/*.wav'
#none_path = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/JUNE_02_BACKGROUND/background_0*.wav'

#uav_path = '../../../RW_AUDIO_DATA_2018_Update/JUNE_01_PHANTOMS/*.wav'
#none_path = '../../../RW_AUDIO_DATA_2018_Update/JUNE_02_BACKGROUND/background_0*.wav'

uav_files = glob.glob(uav_path)
none_files = glob.glob(none_path)
#print(uav_files)
#print(none_files)

In [38]:
CHUNK_SIZE = 8192
SR = 44100
N_MFCC = 16

In [39]:
def load(files, sr=44100):
    [raw, sr] = librosa.load(files[0], sr=sr)
    for f in files[1:]:
        [array, sr] = librosa.load(f, sr=sr)
        raw = np.hstack((raw, array))
    print(raw.shape)
    return raw

In [40]:
uav_raw = load(uav_files)
none_raw = load(none_files)

(884736,)
(884736,)


In [41]:
def mfcc4(raw, label, chunk_size=8192, window_size=4096, sr=44100, n_mfcc=16, n_frame=16):
    mfcc = np.empty((0, n_mfcc, n_frame))
    y = []
    print(raw.shape)
    for i in range(0, len(raw), chunk_size//2):
        mfcc_slice = librosa.feature.mfcc(raw[i:i+chunk_size], sr=sr, n_mfcc=n_mfcc) #n_mfcc,17
        if mfcc_slice.shape[1] < 17:
            print("small end:", mfcc_slice.shape)
            continue
        mfcc_slice = mfcc_slice[:,:-1]
        mfcc_slice = mfcc_slice.reshape((1, mfcc_slice.shape[0], mfcc_slice.shape[1]))
        mfcc = np.vstack((mfcc, mfcc_slice))
        y.append(label)
    y = np.array(y)
    return mfcc, y

In [42]:
mfcc_uav, y_uav = mfcc4(uav_raw, 1)
print(mfcc_uav.shape, y_uav.shape)
mfcc_none, y_none = mfcc4(none_raw, 0)
print(mfcc_none.shape, y_none.shape)


(884736,)
small end: (16, 9)
(215, 16, 16) (215,)
(884736,)
small end: (16, 9)
(215, 16, 16) (215,)


In [43]:
X = np.concatenate((mfcc_uav, mfcc_none), axis=0)
y = np.hstack((y_uav, y_none))
print(X.shape, y.shape)

(430, 16, 16) (430,)


In [44]:
n_labels = y.shape[0]
n_unique_labels = 2
y_encoded = np.zeros((n_labels, n_unique_labels))
y_encoded[np.arange(n_labels), y] = 1
print(y_encoded.shape)

(430, 2)


In [45]:
X.shape

(430, 16, 16)

In [46]:
y_encoded.shape

(430, 2)

In [47]:
X_train2 = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1) ############# 리셰입 해야 
 

In [48]:
X_train2.shape

(430, 16, 16, 1)

In [49]:
with tf.Session() as sess:
    #sess.run(init)
    saver.restore(sess, model_path)
    y_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_train2})
    y_true = sess.run(tf.argmax(y_encoded,1))
print(y_pred.shape,y_true.shape)

INFO:tensorflow:Restoring parameters from C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/GIt/uav-audio-detection/models/CNN/my_test_model_cnn


TypeError: unhashable type: 'numpy.ndarray'

In [58]:
model_path_f = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/GIt/uav-audio-detection/models/CNN/'
filename = 'my_test_model_cnn.meta'


sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
loader = tf.train.import_meta_graph(model_path_f+filename)
loader.restore(sess, tf.train.latest_checkpoint(model_path_f))

SR = 44100
####
justone = True
while(justone):
    justone = False
    y_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_train2})
    y_true = sess.run(tf.argmax(y_encoded,1))

sess.close()

INFO:tensorflow:Restoring parameters from C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/GIt/uav-audio-detection/models/CNN/my_test_model_cnn


C:\Users\Jaesung\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [59]:
print(type(X_train2))

<class 'numpy.ndarray'>


In [60]:
X_train2.shape

(430, 16, 16, 1)

In [61]:
print(type(X_test2))

<class 'numpy.ndarray'>


In [62]:
X_test2.shape

(1382, 16, 16, 1)

In [63]:
from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_true, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

F-Score: 0.5
Accuracy:  0.5
             precision    recall  f1-score   support

          0       0.00      0.00      0.00       215
          1       0.50      1.00      0.67       215

avg / total       0.25      0.50      0.33       430

[[  0 215]
 [  0 215]]


C:\Users\Jaesung\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Experiment 4 - Two convolutional layers with no dropout

#### Experiment 4-2
- learning rate 0.005
- pooling stride 1x1
- #filter 5 for each conv layer
- detects everything as no uav
- no dropout
- batch_size = 100

In [27]:
tf.reset_default_graph()

In [28]:
n_mfcc = 16
n_frame = 16
n_classes = 2
n_channels = 1

kernel_size = 3
stride = 1
pad = "SAME"

learning_rate = 0.005
training_epochs = 20

In [29]:
X = tf.placeholder(tf.float32, shape=[None,n_mfcc*n_frame*n_channels])
X = tf.reshape(X, [-1, n_mfcc, n_frame, n_channels])
Y = tf.placeholder(tf.float32, shape=[None,n_classes])

In [30]:
conv1 = tf.layers.conv2d(inputs=X, filters=5, kernel_size=[3, 3],
                         padding="SAME", activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                                padding="SAME", strides=1)

conv2 = tf.layers.conv2d(inputs=pool1, filters=5, kernel_size=[3, 3],
                         padding="SAME", activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],
                                padding="SAME", strides=2)


In [31]:
flat = tf.reshape(pool2, [-1,  8*8*5])
dense3 = tf.layers.dense(inputs=flat, units=625, activation=tf.nn.relu)
logits = tf.layers.dense(inputs=dense3, units=2)

In [32]:
print(logits)

Tensor("dense_1/BiasAdd:0", shape=(?, 2), dtype=float32)


In [33]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [34]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [35]:
X_train2 = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test2 = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

In [36]:
batch_size = 100
cost_history = np.empty(shape=[1], dtype=float)

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(y_train.shape[0] / batch_size)
    for i in range(0, y_train.shape[0], batch_size):
        feed_dict={X:X_train2[i:i+batch_size,:,:,:], Y:y_train[i:i+batch_size,:]}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        cost_history = np.append(cost_history,cost)
        avg_cost += c/total_batch
    print('Epoch:', '%04d' % (epoch+1), 'cost = ', '{:.9f}'.format(avg_cost))

Epoch: 0001 cost =  13.772808812
Epoch: 0002 cost =  0.053185626
Epoch: 0003 cost =  0.034964081
Epoch: 0004 cost =  0.026989137
Epoch: 0005 cost =  0.020468747
Epoch: 0006 cost =  0.013496138
Epoch: 0007 cost =  0.012944043
Epoch: 0008 cost =  0.012228660
Epoch: 0009 cost =  0.010845582
Epoch: 0010 cost =  0.007515183
Epoch: 0011 cost =  0.005247544
Epoch: 0012 cost =  0.009526652
Epoch: 0013 cost =  0.016303025
Epoch: 0014 cost =  0.007519471
Epoch: 0015 cost =  0.004426770
Epoch: 0016 cost =  0.007495691
Epoch: 0017 cost =  0.016720024
Epoch: 0018 cost =  0.010475561
Epoch: 0019 cost =  0.005024804
Epoch: 0020 cost =  0.003715448


In [37]:
y_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_test2})
y_true = sess.run(tf.argmax(y_test,1))

In [38]:
# cost history 5.75->0.64... but low f1 score (overfitting???)
from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_true, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

F-Score: 0.991
Accuracy:  0.9913169319826338
             precision    recall  f1-score   support

          0       0.99      0.99      0.99       885
          1       0.99      0.99      0.99       497

avg / total       0.99      0.99      0.99      1382

[[880   5]
 [  7 490]]
